In [3]:
from typing import Dict, Union, Any, List

from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, BaseMessage
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI


# Define custom callback handler implementations
class StreamHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        print(f"[Streaming] New token received: {token}")


class DebugHandler(BaseCallbackHandler):
    def on_chat_model_start(
        self,
        serialized: Dict[str, Any],
        messages: List[List[BaseMessage]],
        **kwargs: Any,
    ) -> Any:
        print(f"[Debug] Chat Model Start event: {serialized}")

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        print(f"on_agent_action: {action}")


# Instantiate the handlers
stream_handler = StreamHandler()
debug_handler = DebugHandler()

# Setup the agent. Only the `llm` will issue callbacks for handler2
llm = ChatOpenAI(temperature=0, streaming=True, callbacks=[stream_handler])
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [4]:
agent.run("What is 10 squared?")

[Streaming] New token received: 
[Streaming] New token received: I
[Streaming] New token received:  need
[Streaming] New token received:  to
[Streaming] New token received:  calculate
[Streaming] New token received:  the
[Streaming] New token received:  square
[Streaming] New token received:  of
[Streaming] New token received:  
[Streaming] New token received: 10
[Streaming] New token received: .

[Streaming] New token received: Action
[Streaming] New token received: :
[Streaming] New token received:  Calculator
[Streaming] New token received: 

[Streaming] New token received: Action
[Streaming] New token received:  Input
[Streaming] New token received: :
[Streaming] New token received:  
[Streaming] New token received: 10
[Streaming] New token received: ^
[Streaming] New token received: 2
[Streaming] New token received: 
[Streaming] New token received: 
[Streaming] New token received: ```
[Streaming] New token received: text
[Streaming] New token received: 

[Streaming] New token rece

'The square of 10 is 100.'

In [5]:
agent.run("What is 10 squared?", callbacks=[debug_handler])

[Debug] Chat Model Start event: {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'], 'kwargs': {'temperature': 0.0, 'streaming': True, 'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}}}
[Streaming] New token received: 
[Streaming] New token received: I
[Streaming] New token received:  need
[Streaming] New token received:  to
[Streaming] New token received:  calculate
[Streaming] New token received:  the
[Streaming] New token received:  square
[Streaming] New token received:  of
[Streaming] New token received:  
[Streaming] New token received: 10
[Streaming] New token received: .

[Streaming] New token received: Action
[Streaming] New token received: :
[Streaming] New token received:  Calculator
[Streaming] New token received: 

[Streaming] New token received: Action
[Streaming] New token received:  Input
[Streaming] New token received: :
[Streaming] New token received:  
[Streaming] New token received: 10
[Streaming] New to

'100'

---


## Token Usage


In [15]:
import asyncio
from langchain.callbacks import get_openai_callback
from langchain.schema import SystemMessage

In [16]:
chat = ChatOpenAI()

In [17]:
with get_openai_callback() as cb:
    chat([SystemMessage(content="My name is James")])
total_tokens = cb.total_tokens
print(total_tokens)
assert total_tokens > 0

25


In [18]:
with get_openai_callback() as cb:
    chat([SystemMessage(content="My name is James")])
    chat([SystemMessage(content="My name is James")])
assert cb.total_tokens == total_tokens * 2

In [28]:
await chat.agenerate(
    [
        [SystemMessage(content="Is the meaning of life 42?")],
        [SystemMessage(content="Is the meaning of life 42?")],
    ],
)

LLMResult(generations=[[ChatGeneration(text='The meaning of life being 42 is a humorous reference from Douglas Adams\' science fiction series "The Hitchhiker\'s Guide to the Galaxy." In the story, a supercomputer named Deep Thought determines that the answer to the ultimate question of life, the universe, and everything is indeed 42. However, it is important to note that this is purely a fictional and comedic concept and does not hold any actual philosophical or existential significance outside of the context of the book. The true meaning of life is a subjective and complex philosophical question that varies depending on individual beliefs, values, and perspectives.', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='The meaning of life being 42 is a humorous reference from Douglas Adams\' science fiction series "The Hitchhiker\'s Guide to the Galaxy." In the story, a supercomputer named Deep Thought determines that the answer to the ultimate question of life, the u

In [35]:
# Async callbacks:
with get_openai_callback() as cb:
    await asyncio.gather(
        chat.agenerate(
            [
                [SystemMessage(content="Is the meaning of life 42?")],
                [SystemMessage(content="Is the meaning of life 42?")],
            ],
        )
    )
print(cb.__dict__)

{'successful_requests': 2, 'total_cost': 0.00037900000000000005, 'total_tokens': 197, 'prompt_tokens': 30, 'completion_tokens': 167}
